In [1]:
import pandas as pd

In [2]:
def read_files_into_df(input_filename, output_filename):
    df_input = pd.read_csv(input_filename, names=['packet_in', 'time_in', 'initial'])
    df_input['message'] = df_input['packet_in'].apply(lambda x : int(x[:48], 2))

    df_output = pd.read_csv(output_filename, names=['packet_out', 'time_out', 'final'])
    df_output['message'] = df_output['packet_out'].apply(lambda x : int(x[:48], 2))

    joined_df = df_input.join(df_output, on='message', lsuffix='left')
    joined_df = joined_df.drop(columns=['messageleft', 'packet_in', 'packet_out'])
    joined_df = joined_df.dropna()
    joined_df['time_in'] = joined_df['time_in'].apply( lambda x : int(x.split(' ')[0]) )
    joined_df['time_out'] = joined_df['time_out'].apply( lambda x : int(x.split(' ')[0]) )
    joined_df['difference'] = joined_df['time_out']-joined_df['time_in']

    return joined_df

data = read_files_into_df('mesh_input_2.txt', 'mesh_output_2.txt')
data

,time_in,initial,time_out,final,message,difference
0,5000,(0.0),80000,(0.1),0,75000
1,11000,(0.0),156000,(0.1),1,145000
2,49000,(0.0),232000,(0.1),2,183000
3,125000,(0.0),308000,(0.1),3,183000
4,201000,(0.0),384000,(0.1),4,183000
...,...,...,...,...,...,...
125995,1259008578406000,(9.9),1259008578589000,(9.8),125995,183000
125996,1259008578482000,(9.9),1259008578665000,(9.8),125996,183000
125997,1259008578558000,(9.9),1259008578741000,(9.8),125997,183000
125998,1259008578634000,(9.9),1259008578817000,(9.8),125998,183000


In [5]:
def get_detailed_info(initial_coords: tuple, dataframe: pd.DataFrame):
    initial_str = f"({initial_coords[0]}.{initial_coords[1]})"

    df = dataframe[dataframe["initial"] == initial_str]

    # first message travelling in the system
    table = df[df["message"] % 10 == 0][["initial", "final", "difference"]]
    table = table.rename(columns={"difference": "first-diff"})

    # second message travelling in the system
    sec = df[df["message"] % 10 == 1][["initial", "final", "difference"]]
    table = table.merge(sec, on=["final", "initial"]).rename(columns={"difference": "second-diff"})


    # last message travelling in the system
    last = df[df["message"] % 10 == 9][["initial", "final", "difference"]]
    table = table.merge(last, on=["final", "initial"]).rename(columns={"difference": "last-diff"})

    # to find the number of messages before it stabilizes
    max_value = df.groupby("final")["difference"].max()
    # basically the same as `table['last-diff']` ^
    df_and_max_value = pd.merge(df, max_value, on='final')
    df_only_not_max_value = df_and_max_value[df_and_max_value["difference_y"] != df_and_max_value["difference_x"]]
    count = df_only_not_max_value.groupby('final').count()["difference_y"]
    table = pd.merge(count, table, on=["final"]).rename(columns={"difference_y": "msgs-until-full"})
    #table = table.unique()
    table.drop_duplicates(subset=["final"], keep="first",inplace=True)

    return table

info_00 = get_detailed_info((0,0), data)
info_00["first-diff"] = info_00["first-diff"]/100
info_00["last-diff"] = info_00["last-diff"]/100

info_00.to_latex(index=False, columns=["initial", "first-diff", "last-diff"])


,final,msgs-until-full,initial,first-diff,second-diff,last-diff
0,(0.1),2,(0.0),75000,145000,183000
1000,(0.2),7,(0.0),119000,189000,308000
2000,(0.3),12,(0.0),163000,233000,390000
3000,(0.4),17,(0.0),207000,277000,434000
4000,(0.5),22,(0.0),251000,321000,478000
5000,(0.6),26,(0.0),295000,365000,522000
6000,(0.7),31,(0.0),339000,409000,566000
7000,(0.8),36,(0.0),383000,453000,610000
8000,(0.9),2,(0.0),402000,455000,476000
9000,(1.0),2,(0.0),75000,145000,183000


In [28]:
# info01 = get_detailed_info((0,1), data) 
# info01
# weird data ^

info10 = get_detailed_info((1,0), data)
info10

,final,msgs-until-full,initial,first-diff,second-diff,last-diff
0,(0.0),1,(1.0),75000,137000,137000
1000,(0.1),2,(1.0),68000,121000,129000
2000,(0.2),7,(1.0),125000,195000,301000
3000,(0.3),12,(1.0),169000,239000,383000
4000,(0.4),17,(1.0),213000,283000,427000
5000,(0.5),22,(1.0),257000,327000,471000
6000,(0.6),26,(1.0),301000,371000,515000
7000,(0.7),31,(1.0),345000,415000,559000
8000,(0.8),36,(1.0),389000,459000,603000
9000,(0.9),2,(1.0),408000,461000,469000


In [25]:
get_detailed_info((9,9), data)


,initial,final,first-diff,second-diff,last-diff
0,(9.9),(0.0),181000,230000,366000
1000,(9.9),(0.1),226000,292000,564000
2000,(9.9),(0.2),227000,293000,531000
3000,(9.9),(0.3),228000,294000,498000
4000,(9.9),(0.4),229000,295000,465000
5000,(9.9),(0.5),230000,296000,432000
6000,(9.9),(0.6),231000,297000,399000
7000,(9.9),(0.7),232000,298000,366000
8000,(9.9),(0.8),233000,299000,333000
9000,(9.9),(0.9),402000,455000,476000
